# Universidad del Valle de Guatemala
## Facultad de Ingeniería
### Departamento de Computación

---

# Proyecto 2: Resultados Iniciales
### Reto 04: Predicción de compradores recurrentes: cuestionar la línea base - Negocios

**Integrantes:**
- Diego Alexander Hernández Silvestre, 21270
- Linda Inés Jiménez Vides, 21169
- Mario Antonio Guerra Morales, 21008
- David Jonathan Aragon Vasquez, 21053

**Curso:** Data Science  
**Sección:** 10  
**Grupo** 4  

---

Guatemala, 28 de octubre de 2024


##### 🫳 Para la descarga del dataset de este reto, es necesario la creación de una cuenta dentro de Alibaba Cloud. Para facilitar este proceso se proporcionan los elementos a descargar en estos enlaces:
- Data_Format1: https://drive.google.com/file/d/1jgsaSM0wypPMUWaNlXy5s_kO0uiXNzES/view?usp=sharing
- Data_Format2: https://drive.google.com/file/d/1dnMeaon-ovASN2HEI1WSNfnw-7lyINAh/view?usp=drivesdk
- sample_submission.csv: 

In [2]:
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import matplotlib.ticker as ticker
from imblearn.over_sampling import SMOTE

### 📩 Carga de los datos

In [10]:
dataTrain = pd.read_csv('data/dataTrain_balanced_timestamp.csv')

In [11]:
dataTrain.head()

,Unnamed: 0.1,user_id,age_range,gender,merchant_id,item_id,category_id,brand_id,clicks,add_to_cart,purchases,add_to_favorites,total_actions,unique_item_count,label,time_stamp
0,0,34176,6.0,0.0,944,408895,1505,7370,1,0,0,0,1,1,-1,"DatetimeIndex(['2023-11-07'], dtype='datetime6..."
1,1,34176,6.0,0.0,412,17235,1604,4396,8,0,0,0,8,7,-1,"DatetimeIndex(['2023-08-18', '2023-08-18', '20..."
2,2,34176,6.0,0.0,1945,231901,662,2758,7,0,0,0,7,3,-1,"DatetimeIndex(['2023-08-18', '2023-08-18', '20..."
3,3,34176,6.0,0.0,4752,174142,821,6938,1,0,0,0,1,1,-1,"DatetimeIndex(['2023-10-27'], dtype='datetime6..."
4,4,34176,6.0,0.0,643,716371,1505,968,0,0,0,1,1,1,-1,"DatetimeIndex(['2023-10-24'], dtype='datetime6..."


In [12]:
dataTrain.info()
dataTrain.count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20300811 entries, 0 to 20300810
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0.1       int64  
 1   user_id            int64  
 2   age_range          float64
 3   gender             float64
 4   merchant_id        int64  
 5   item_id            int64  
 6   category_id        int64  
 7   brand_id           int64  
 8   clicks             int64  
 9   add_to_cart        int64  
 10  purchases          int64  
 11  add_to_favorites   int64  
 12  total_actions      int64  
 13  unique_item_count  int64  
 14  label              int64  
 15  time_stamp         object 
dtypes: float64(2), int64(13), object(1)
memory usage: 2.4+ GB


Unnamed: 0.1         20300811
user_id              20300811
age_range            20300811
gender               20300811
merchant_id          20300811
item_id              20300811
category_id          20300811
brand_id             20300811
clicks               20300811
add_to_cart          20300811
purchases            20300811
add_to_favorites     20300811
total_actions        20300811
unique_item_count    20300811
label                20300811
time_stamp           20300811
dtype: int64

In [13]:
dataTrain.describe()

,Unnamed: 0.1,user_id,age_range,gender,merchant_id,item_id,category_id,brand_id,clicks,add_to_cart,purchases,add_to_favorites,total_actions,unique_item_count,label
count,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07
mean,3.513824e+06,2.116874e+05,3.037811e+00,3.359498e-01,2.512599e+03,5.559079e+05,8.667681e+02,4.163890e+03,8.630119e+00,5.517021e-06,9.027854e-01,3.033002e-01,1.008873e+01,4.353554e+00,0.000000e+00
std,2.027742e+06,1.215602e+05,1.710790e+00,4.892762e-01,1.287225e+03,3.217158e+05,3.902042e+02,2.163676e+03,2.279894e+01,2.410922e-03,8.730307e-01,1.382527e+00,2.359851e+01,8.344923e+00,8.164966e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,-1.000000e+00
25%,1.765179e+06,1.056220e+05,2.000000e+00,0.000000e+00,1.480000e+03,2.750960e+05,5.960000e+02,2.368000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,-1.000000e+00
50%,3.513432e+06,2.122770e+05,3.000000e+00,0.000000e+00,2.492000e+03,5.548880e+05,8.490000e+02,4.073000e+03,4.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,5.000000e+00,2.000000e+00,0.000000e+00
75%,5.261091e+06,3.166630e+05,4.000000e+00,7.078987e-01,3.596000e+03,8.307000e+05,1.180000e+03,5.910000e+03,9.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.100000e+01,4.000000e+00,1.000000e+00
max,7.030722e+06,4.241700e+05,8.000000e+00,2.000000e+00,4.995000e+03,1.113166e+06,1.671000e+03,8.477000e+03,6.962000e+03,2.000000e+00,2.020000e+02,3.980000e+02,6.963000e+03,1.194000e+03,1.000000e+00
